# 🚌 TR-01: Transporte masivo (LATAM) – GTFS/GTFS-RT y métricas operativas

Este cuaderno guía la ingestión y análisis de datos GTFS/GTFS-RT aplicados al contexto LATAM, con foco en validación de esquema, procesamiento geoespacial, métricas de operación y visualizaciones.

## 1) Configurar entorno y dependencias

- Librerías: `pandas`, `geopandas`, `numpy`, `matplotlib`, `folium`/`leafmap`, `partridge` o `gtfs_kit`, `pytz`, `plotly`.
- Instalar en el venv del repo (VS Code Terminal):
```pwsh
F:\GitHub\supply-chain-data-notebooks\.venv\Scripts\python.exe -m pip install geopandas folium leafmap partridge pytz plotly
```
- Cargar librerías en las celdas de código siguientes.

In [1]:
# Placeholder: importación de librerías (ajuste según tu entorno)
import pandas as pd
import numpy as np

## 2) Cargar y validar paquete GTFS (esquema)
Archivos esperados: `agency.txt`, `routes.txt`, `trips.txt`, `stops.txt`, `stop_times.txt`, `shapes.txt`, `calendar.txt` y/o `calendar_dates.txt`. Validar presencia y tipos de columnas clave (`route_id`, `trip_id`, `stop_id`).

In [2]:
# Placeholder: lectura GTFS (sin implementar)
# Sugerencia: usar partridge o gtfs_kit para leer el ZIP y acceder a tablas.
gtfs_zip_path = "path/a/tu_feed_gtfs.zip"
# TODO: implementar lectura con partridge/gtfs_kit y validaciones de columnas obligatorias.

## 3) Normalizar y limpiar datos para LATAM
- Normalizar encoding UTF-8 y nombres de `agency`, resolver zonas horarias (`America/Santiago`, `America/Mexico_City`, `America/Bogota`).
- Mapear `route_type` locales a estándar GTFS.
- Unificar formatos de `stop_id`/`trip_id`.
- Integrar feriados locales desde `calendar_dates.txt`.

## 4) Construir tablas relacionales: agency, routes, trips, stops, shapes
- Unir `trips` con `routes` y `shapes`.
- Generar índices y columnas para análisis (ej. `servicio_diario`, `sentido`, `vehiculo_tipo`).
- Snippets para joins eficientes con `pandas`/`geopandas`.

## 5) Generar geometrías y GeoDataFrames (paradas y trazas)
- Crear `GeoDataFrame` de paradas (Point) y trazas de ruta (LineString) a partir de `shapes.txt`.
- Proyectar a CRS métricos (ej. EPSG:3857 / CRS locales).
- Simplificación de trazas y cálculo de distancias entre paradas.

## 6) Calcular métricas operativas básicas (headway, span, trips/día)
- Headway por línea/segmento: media de diferencias entre salidas, $h=\mathrm{mean}(\Delta t)$.
- Span de servicio: primer/último viaje.
- Trips/día por `route_id` y cobertura espacial.

## 7) Puntualidad y desempeño usando GTFS-RT o simulación
- Ingesta básica de GTFS-RT (posiciones) o simulación de retrasos.
- Vincular con `trips` GTFS y calcular puntualidad.
- OTP: $$\mathrm{OTP}=\frac{N_{\text{on-time}}}{N_{\text{total}}}$$, umbrales típicos ±5 min; métricas por hora/segmento.

In [3]:
# Placeholder: GTFS-RT (sin implementar)
gtfs_rt_endpoint = "url/a/gtfs-rt.pb"
# TODO: fetch y parse de protobuf/JSON, vínculo con trips y cálculo de puntualidad.

## 8) Estimación de capacidad y niveles de servicio
- Capacidad por viaje usando tablas de vehículo o supuestos.
- Cálculo de **load factor** y nivel de servicio por parada y por hora.
- Métricas: pasajeros estimados / capacidad, frecuencia efectiva.

## 9) Visualizaciones interactivas y paneles
- Mapas con `folium`/`leafmap` para rutas/paradas.
- Series temporales (headways, OTP) con `plotly`.
- Heatmaps de cobertura y demanda estimada.
- Exportar HTML interactivo.

## 10) Exportar métricas y reportes
- Guardar tablas resumen en CSV/JSON y GeoPackage.
- Comandos para escribir en disco y visualizar resultados en VS Code Output.

## 11) Pruebas unitarias y verificación en VS Code
- Tests con `pytest` para funciones críticas: validación de esquema, cálculo de headway, vínculo GTFS-RT.
- Ejecutar en el runner integrado y revisar el panel de pruebas.

## 📎 Feeds GTFS públicos (referencia LATAM)
- Chile (Santiago): https://www.dtpm.cl/index.php/gtfs-transantiago/ 
- México (CDMX): https://datos.cdmx.gob.mx/explore/?sort=modified&tags=gtfs 
- Brasil (São Paulo/EMTU/Metrô): https://www.sptrans.com.br/desenvolvedores/ e https://www.metro.sp.gov.br/data/gtfs/ 
- Colombia (Bogotá): https://datosabiertos.bogota.gov.co/ (buscar GTFS) 

Puedes usar cualquiera de estos para pruebas locales.

In [ ]:
# 🔽 Descarga de un GTFS de muestra (local)
# Nota: reemplaza la URL por una real del listado anterior.
import os
from pathlib import Path
import requests

root = Path(__file__).resolve().parents[2]
data_dir = root / 'data' / 'raw'
data_dir.mkdir(parents=True, exist_ok=True)

sample_url = 'https://gtfs.zip.example/your_feed.zip'  # TODO: pon una URL válida
out_zip = data_dir / 'gtfs_sample.zip'

if not out_zip.exists():
    try:
        r = requests.get(sample_url, timeout=60)
        r.raise_for_status()
        out_zip.write_bytes(r.content)
        print(f"Descargado: {out_zip}")
    except Exception as e:
        print("No se pudo descargar el GTFS de muestra. Usa uno local.", e)

## 📄 Lectura GTFS con partridge/gtfs_kit (mínimo)
Intentamos cargar el ZIP local `data/raw/gtfs_sample.zip` leyendo tablas clave (`stops`, `routes`, `trips`). Si no están instalados, se muestra un mensaje con instrucciones.

In [ ]:
# Intento de lectura con partridge; si falla, probar gtfs_kit
from pathlib import Path
zip_path = Path(__file__).resolve().parents[2] / 'data' / 'raw' / 'gtfs_sample.zip'

stops = routes = trips = None

try:
    import partridge as pt
    feed = pt.load_feed(zip_path)
    stops = feed.stops
    routes = feed.routes
    trips = feed.trips
    print("Leído con partridge:", len(stops), "paradas;", len(routes), "rutas;", len(trips), "trips")
except Exception as e:
    print("partridge no disponible o fallo:", e)
    try:
        import gtfs_kit as gk
        import zipfile
        with zipfile.ZipFile(zip_path) as zf:
            stops = gk.read_table(zf, 'stops')
            routes = gk.read_table(zf, 'routes')
            trips = gk.read_table(zf, 'trips')
        print("Leído con gtfs_kit:", len(stops), "paradas;", len(routes), "rutas;", len(trips), "trips")
    except Exception as e2:
        print("gtfs_kit no disponible o fallo:", e2)
        print("Instala dependencias:")
        print("F:/GitHub/supply-chain-data-notebooks/.venv/Scripts/python.exe -m pip install partridge gtfs_kit")

## 🗺️ Mapa rápido de paradas (folium)
Si se cargaron `stops`, creamos un mapa sencillo centrado en la primera parada y lo guardamos como HTML.

In [ ]:
# Generar mapa si hay stops disponibles
try:
    import folium
    if stops is not None and len(stops) > 0:
        # detectar nombres de columnas lat/lon
        lat_col = 'stop_lat' if 'stop_lat' in stops.columns else None
        lon_col = 'stop_lon' if 'stop_lon' in stops.columns else None
        if lat_col and lon_col:
            center = [float(stops.iloc[0][lat_col]), float(stops.iloc[0][lon_col])]
            m = folium.Map(location=center, zoom_start=12)
            for _, row in stops.head(200).iterrows():
                try:
                    folium.CircleMarker(
                        location=[float(row[lat_col]), float(row[lon_col])],
                        radius=3,
                        color='blue',
                        fill=True,
                        fill_opacity=0.7,
                        popup=str(row.get('stop_name', 'stop'))
                    ).add_to(m)
                except Exception:
                    pass
            out_html = (Path(__file__).resolve().parents[2] / 'data' / 'processed' / 'gtfs_stops_map.html')
            out_html.parent.mkdir(parents=True, exist_ok=True)
            m.save(str(out_html))
            print("Mapa guardado:", out_html)
        else:
            print("No se encontraron columnas stop_lat/stop_lon en stops.")
    else:
        print("No hay stops cargados para mapear.")
except Exception as e:
    print("No se pudo generar el mapa:", e)